# Exemple d'utilisation d'un transfcormer dans le cadre de la génération de code.

On utilise au chois les modeles

* codegen-350M-mono
* codegen-2B-mono
*

In [1]:
model_name = "export_model/codegen-350M-mono_finetuned"

# Observations

* En promptant sur un debut de code le model est capable de prédire du code correcte sur des soucis simple mais pas parfait
* En utilisant le model il semble y avoir conservation du context
    * Pour vérifier réinitialisé l'appel entre chaque

## Initialisation du model

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
# Création du tokenniser
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Chargement du model
model = AutoModelForCausalLM.from_pretrained(model_name)

# On declare en global (beurk !) le device pour tester
device = None

# Vérifie si le GPU est disponible avant de continuer, on met alors à jour device
if torch.cuda.is_available():
    # Sélectionne spécifiquement le premier GPU (device 0)
    device = torch.device("cuda:0")
    print("Utilisation du GPU :", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU non trouvé, utilisation du CPU.")

# Déplace le modèle vers le GPU 0
# ca plante !
model.to(device)


Utilisation du GPU : NVIDIA GeForce RTX 3080 Ti


CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [5]:
# Parametres du model

max_length=200,  # Adjust as needed
num_beams=5,      # Adjust for quality/speed trade-off
temperature=0.7,  # Adjust for creativity (higher = more creative)
top_k=40,         # Adjust for sampling
top_p=0.95,        # Adjust for sampling
pad_token_id=tokenizer.eos_token_id # Important for some models

In [6]:
def do_predict(model,
               prompt: str,
               max_length: int = 200,
               temperature: float = 0.7,
               top_k: int = 40,
               top_p: float = 0.9,
               pad_token_id: int = tokenizer.eos_token_id):
    print(f"Prompte : {prompt}\n----")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # On doit transférer les sequence de token dans le GPU.
    input_ids = input_ids.to(device)

    print(f"Input ids : {input_ids}")
    print("---")
    generated_ids = model.generate(input_ids,
                                   max_length=max_length,
                                   temperature=temperature,
                                   top_k=top_k,
                                   top_p=top_p,
                                   pad_token_id=pad_token_id,
                                   do_sample=True)
    print("----")
    print(generated_ids[0])
    print("----")
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


## Utilisation pour faire de la completion de code

On fournis un fragment de code et on demande au modele de compléter.

In [7]:
text = """
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:
"""

print(do_predict(model, text))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]], device='cuda:0')
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    64,    92,  1556, 38251, 13528, 28141,  1391,    65,
        36786,   198,     8,   198,   198,   361,   257,  6624,   275,    25,
          198, 50284,  4798,     7,    69,     1,    90,    64,    92,  1556,
        38251, 13528, 28141,  1391,    65, 3

In [8]:
print(do_predict(model, text, temperature=1))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]], device='cuda:0')
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    64,    92,  1556, 38251, 13528, 28141,  1391,    65,
        36786,   198,     8,   198,   198,   361,   257,  6624,   275,    25,
          198, 50284,  4798,     7,    69,     1,    90,    64,    92,  1556,
        38251, 13528, 28141,  1391,    65, 3

In [9]:
print(do_predict(model, text, temperature=0.01))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]], device='cuda:0')
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    64,    92,  1556, 38251, 13528, 28141,  1391,    65,
        36786,   198,     8,   198,   198,   361,   257,  6624,   275,    25,
          198, 50284,  4798,     7,    69,     1,    90,    64,    92,  1556,
        38251, 13528, 28141,  1391,    65, 3

In [10]:
print(do_predict(model, text, temperature=0.01, max_length=1024))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]], device='cuda:0')
---
----
tensor([  198,    64,   796,  ..., 13528, 28141,  1391], device='cuda:0')
----

a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"
)

if a == b:
    print(f"{a} est égal à {b}"

In [14]:
print(do_predict(model, """
genere une fonction pour calculer la somme des 5 premiers chiffres
"""))

Prompte : 
genere une fonction pour calculer la somme des 5 premiers chiffres

----
Input ids : tensor([[  198,  5235,   567, 17809,   277,   261,   596, 12797,  5204,   263,
          8591,  3870,  1326,   748,   642,  4199,  3183,   442,   733,   411,
           198]], device='cuda:0')
---
----
tensor([  198,  5235,   567, 17809,   277,   261,   596, 12797,  5204,   263,
         8591,  3870,  1326,   748,   642,  4199,  3183,   442,   733,   411,
          198,  4299,  3870,  1326,    62,   354,   733,   411,     7,    77,
         2381,   411,  2599,   198, 50284,  7783,  2160,     7,   600,     7,
          354,   733,   260,     8,   329,   442,   733,   260,   287,   965,
            7,    77,  2381,   411,  4008,   198,   198,  4798,     7,    82,
          296,  1326,    62,   354,   733,   411,     7,    77,  2381,   411,
         4008,   198,   198,  4299,  3870,  1326,    62,   354,   733,   411,
            7,    77,  2381,   411,  2599,   198, 50284,  7783,  2160,     7,


## test du code

### 1er run

Résultant sacrément mauvais par rapport au model de base non finetunné.

In [15]:
# (ajout du # car manquant dans la génération) genere une fonction pour calculer la somme des 5 premiers chiffres
def somme_chiffres(nombres):
    return sum(int(chiffre) for chiffre in str(nombres))
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)
return somme_chiffres(nombres)

SyntaxError: 'return' outside function (1825139391.py, line 4)

### 2eme run

Pas mieux

In [16]:
# (ajout du # car manquant dans la génération) genere une fonction pour calculer la somme des 5 premiers chiffres
def somme_chiffres(nombres):
    return sum(nombres)

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(nombres)

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(nombres)

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(nombres)

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(nombres)

NameError: name 'nombres' is not defined

3eme run

In [17]:
# (ajout du # car manquant dans la génération) genere une fonction pour calculer la somme des 5 premiers chiffres
def somme_chiffres(nombres):
    return sum(int(chiffre) for chiffre in str(nombres))

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(int(chiffre) for chiffre in str(nombres))

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(int(chiffre) for chiffre in str(nombres))

print(somme_chiffres(nombres))

def somme_chiffres(nombres):
    return sum(int(chiffre)

SyntaxError: incomplete input (1080615747.py, line 18)

# Bilan

Le model fine tunné est bien pire que le model de base, on voit que le format des données en input est trés important.

le dataset structuré "prompt\ncode" semble contre productif car le code généré oublis les # pour marqué un commentaires, le code tourne en boucle.

Il est super important de préparer le dataset et il semble aussi important de l'interroger dans un format bien définie.

## Poursuite des investigations

Voir :
* [ ] comment faire un bon dataset
* [ ] interoger correctement le model
*